In [1]:
import pandas as pd
selected_columns = ["id","model","domain","target"]
testing_frame = pd.read_parquet("/kaggle/input/dactyl-adversarial/finetuned-texts.parquet")

testing_frame = testing_frame[testing_frame.split == "testing"]

In [2]:
# Use a pipeline as a high-level helper
from transformers import pipeline
from tqdm import tqdm
pipe = pipeline("text-classification", model="yaful/MAGE")

def data():
    for text in testing_frame["text"].values:
        yield text
y_pred = list()
for result in tqdm(pipe(data(),batch_size=16, max_length=512,truncation=True, return_all_scores=True), total=len(testing_frame)):
    for output in result:
        if output["label"] == 0:
            y_pred.append(output["score"])



config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/595M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/382 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0
/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
  0%|          | 0/5516 [00:00<?, ?it/s]Initializing global attention on CLS token...
Input ids are automatically padded to be a multiple of `config.attention_window`: 512
100%|██████████| 5516/5516 [05:12<00:00, 17.68it/s]


In [3]:
testing_frame["mage_pred"] = y_pred
testing_frame[selected_columns + ["mage_pred"]].to_csv(f"mage-results.csv",index=False)
